Import Libraries

In [204]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Get the Understat_ID of the players from the below link

In [205]:

url = "https://github.com/vaastav/Fantasy-Premier-League/blob/master/data/2022-23/id_dict.csv?raw=true"
df_understat = pd.read_csv(url)

#Clean the column names
df_understat.columns = df_understat.columns.str.replace(' ', '')

#Get the column names
df_understat.columns

Index(['Understat_ID', 'FPL_ID', 'Understat_Name', 'FPL_Name'], dtype='object')

Get the Fpl Data which has been scrapped using FPL API

In [206]:
df_fpl = pd.read_excel('First_Draft.xlsx')

#Get the column names
df_fpl.columns

Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Total Points', 'Bonus', 'Minutes',
       'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Form'],
      dtype='object')

In [207]:
# Merge df_fpl and df_understat using the player_id column and _FPL_id column
df_fpl_understat = pd.merge(df_fpl, df_understat, left_on = "Player ID", right_on = "FPL_ID")

#Get the column names
df_fpl_understat.columns


Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Total Points', 'Bonus', 'Minutes',
       'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Form',
       'Understat_ID', 'FPL_ID', 'Understat_Name', 'FPL_Name'],
      dtype='object')

In [208]:
#keep only Understat_ID 
df_fpl_understat = df_fpl_understat[['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By',
    'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus', 'Minutes',
    'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Understat_ID']]

#convert Understat_ID to int
df_fpl_understat['Understat_ID'] = df_fpl_understat['Understat_ID'].astype(int)

pip install undetstatapi and import UnderstatClient (https://pypi.org/project/understatapi/)

In [209]:
from understatapi import UnderstatClient
understat = UnderstatClient()

#Change Season to 2022 when the season starts on 5th August
league_player_data = understat.league(league="EPL").get_player_data(season="2021")

# convert to pandas dataframe
league_player_data = pd.DataFrame(league_player_data)
league_player_data.columns

Index(['id', 'player_name', 'games', 'time', 'goals', 'xG', 'assists', 'xA',
       'shots', 'key_passes', 'yellow_cards', 'red_cards', 'position',
       'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup'],
      dtype='object')

In [210]:
#convert columns to int
league_player_data['games'] = league_player_data['games'].astype(int)
league_player_data['id'] = league_player_data['id'].astype(int)
league_player_data['xG'] = league_player_data['xG'].astype(float)
league_player_data['xA'] = league_player_data['xA'].astype(float)
league_player_data['npxG'] = league_player_data['npxG'].astype(float)
league_player_data['xGChain'] = league_player_data['xGChain'].astype(float)
league_player_data['xGBuildup'] = league_player_data['xGBuildup'].astype(float)
league_player_data['shots'] = league_player_data['shots'].astype(int)
league_player_data['key_passes'] = league_player_data['key_passes'].astype(int)


In [211]:
#merge with df_merged
final_df= pd.merge(df_fpl_understat, league_player_data, left_on = "Understat_ID", right_on = "id")

In [212]:
final_df.columns

Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus',
       'Minutes', 'Goals Scored', 'Assists', 'Clean Sheets', 'Status',
       'Understat_ID', 'id', 'player_name', 'games', 'time', 'goals', 'xG',
       'assists', 'xA', 'shots', 'key_passes', 'yellow_cards', 'red_cards',
       'position', 'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup'],
      dtype='object')

In [213]:
final_df = final_df[['Player ID', 'Understat_ID','Team', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By',
    'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus', 'games', 'Minutes',
    'Goals Scored', 'xG', 'Assists', 'xA', 'npxG', 'xGChain', 'xGBuildup', 'Clean Sheets', 'Status']]

In [214]:
#all numeric number to round to 2 decimal places
final_df = final_df.round(2)

In [215]:
final_df.columns

Index(['Player ID', 'Understat_ID', 'Team', 'First Name', 'Second Name',
       'Web Name', 'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus',
       'games', 'Minutes', 'Goals Scored', 'xG', 'Assists', 'xA', 'npxG',
       'xGChain', 'xGBuildup', 'Clean Sheets', 'Status'],
      dtype='object')

In [216]:
#rename columns
final_df.columns = ['Player ID', 'Understat_ID','Team', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By', 'Transfers In', 'Transfers Out', 'Form','Total Points', 'Bonus', 'Matches', 'Minutes', 'Goals Scored', 'xG', 'Assists', 'xA', 'npxG','xGChain', 'xGBuildup', 'Clean Sheets', 'Status']

In [217]:
#sort by Selected By and Total Points
final_df = final_df.sort_values(by=['Selected By', 'Total Points'], ascending=False)
final_df.head(10)
import datetime
final_df.to_excel('final_df '+str(datetime.datetime.today().strftime("%d-%m-%Y"))+'.xlsx', index=False)